  FAKE NEWS PREDICTION  

IMPORTING THE DEPENDENCIES

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords # nltk - natural language tool kit  removing the unwanted words
from nltk. stem.porter import PorterStemmer # stemming is the process of removing prefixes and suffixes of the word and returning the root word

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Incerting The Dataset

In [ ]:
fake = pd.read_csv('/content/Fake.csv.zip')
true = pd.read_csv('/content/True.csv.zip')

In [ ]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
fake.info()
true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [ ]:
true.shape

(21417, 4)

In [ ]:
fake.shape

(23481, 4)

In [ ]:
fake.isnull().sum()

,0
title,0
text,0
subject,0
date,0


In [ ]:
true.isnull().sum()

,0
title,0
text,0
subject,0
date,0


In [ ]:
fake['class'] = 0
true['class'] = 1

In [ ]:
fake_manual_testing = fake.tail(10)
fake.drop(fake.tail(10).index, axis=0, inplace=True)
true_manual_testing = true.tail(10)
true.drop(true.tail(10).index,axis=0,inplace=True)

In [ ]:
df_maual_testing = pd.concat([fake_manual_testing, true_manual_testing], axis=0)
df_maual_testing.to_csv('manual_testing.csv')

In [ ]:
df_marge = pd.concat([fake, true], axis=0)
df_marge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [ ]:
df = df_marge.drop(['title', 'subject','date'],axis = 1)
df.head(10)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


In [ ]:
df= df.sample(frac=1)


In [ ]:
df.head(10)

,text,class
1670,WASHINGTON (Reuters) - A U.S. Senate panel pro...,1
610,WASHINGTON (Reuters) - U.S. House of Represent...,1
8234,"The NRA tells us that guns make us safer, that...",0
6806,WASHINGTON (Reuters) - President-elect Donald ...,1
18007,Washington Post: Is this the woman that Michel...,0
21235,BEIRUT (Reuters) - U.S.-led warplanes on Wedne...,1
2058,Just last year before the 2016 presidential el...,0
9447,The New York Times just confirmed what we ve a...,0
22159,21st Century Wire says President Trump is drif...,0
11854,RIYADH (Reuters) - Saudi air defenses shot dow...,1


In [ ]:
df.isnull().sum()

,0
text,0
class,0


In [ ]:
def word_drop(text):
  text =text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub("\\W"," ",text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('\n','',text)
  text = re.sub('\w\d\w*','',text)
  return text

In [ ]:
df['text'] = df['text'].apply(word_drop)

In [ ]:
df.head(10)

,text,class
1670,washington reuters a u s senate panel pro...,1
610,washington reuters u s house of represent...,1
8234,the nra tells us that guns make us safer that...,0
6806,washington reuters president elect donald ...,1
18007,washington post is this the woman that michel...,0
21235,beirut reuters u s led warplanes on wedne...,1
2058,just last year before the presidential electi...,0
9447,the new york times just confirmed what we ve a...,0
22159,century wire says president trump is drifting...,0
11854,riyadh reuters saudi air defenses shot dow...,1


In [ ]:
x = df['text']
y = df['class']


In [ ]:
x_train , x_test, y_train , y_test = train_test_split(x,y, test_size=0.25)

In [ ]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [ ]:
LR.score(xv_test, y_test)

0.9860071301247771

In [ ]:
pred_LR = LR.predict(xv_test)

In [ ]:
print(accuracy_score(y_test, pred_LR))

0.9860071301247771


In [ ]:

print(classification_report( y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5977
           1       0.98      0.99      0.99      5243

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [ ]:
##decion tree classiffication##

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
GCB = GradientBoostingClassifier(random_state=0)

In [ ]:
GCB.fit(xv_test, y_test)

GradientBoostingClassifier(random_state=0)

In [ ]:
pred_GCB = GCB.predict(xv_test)

In [ ]:
GCB.score(xv_test, y_test)

0.9959893048128342

In [ ]:
print(classification_report(y_test, pred_GCB))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5977
           1       0.99      1.00      1.00      5243

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train )

RandomForestClassifier(random_state=0)

In [ ]:
RFC.score(xv_test, y_test)

0.9923351158645276

In [ ]:
pred_RFC = RFC.predict(xv_test)

In [ ]:
print(classification_report(y_test, pred_RFC))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5977
           1       0.99      0.99      0.99      5243

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [ ]:
def output_lable(n):
  if n == 0 :
    return "FAKE NEWS"
  elif n == 1:
    return "NOT A FAKE NEWS"
def manual_testing(news):
  testing_news = {"text": [news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(word_drop)
  new_xv_test = vectorization.transform(new_def_test["text"])
  pred_LR = LR.predict(new_xv_test)
  pred_GCB = GCB.predict(new_xv_test)
  pred_RFC = RFC.predict(new_xv_test)

  return print("\n\nLR Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                              output_lable(pred_GCB[0]),
                                                                                                              output_lable(pred_RFC[0])))

In [ ]:
news = str(input())
manual_testing(news)

21st Century Wire says Al Jazeera America will go down in history as one of the biggest failures in broadcast media history.Ever since the US and its allies began plotting to overthrow Libya and Syria, Al Jazeera has deteriorated from a promising international news network in 2003   into what it has become in 2016   a full-blown agit prop media shop for the US State Department and the Pentagon. In fact, US Military Central Command (CENTCOM) for the Middle East region is located in Qatar s capital of Doha, conveniently located just down the road from Al Jazeera s headquarters. Nice and cozy.Al Jazeera s role in promoting  regime change  and the destablization of Syria is now recorded history, as are its many shameful exhibitions of media fakery.The Qatari royals tried to expand propaganda operations into North America, but no matter how much money they threw at it (and those royals just love throwing money at things to try and get them to work), no one cared about it, much less watched 